# Web Scrapping

## Getting the right libraries

In [9]:
#We get all packages that we will need during our web scraping journey
import unicodecsv as csv #we will need this at the end of the work , to export our Data Base on CSV format
import urllib #this library containt several modules for working with URLs:
from urllib import request #for opening and reading URLs
import bs4 #Beautiful Soup is a Python library for pulling data out of HTML and XML file
from bs4 import BeautifulSoup
import pandas as pd #Data Analysis Library



# Build and initialise the Database structure

In [ ]:


#init of the database object
DataBase=[]
nb_pages=0
i=0
#the path to your download content (to replace for yout path)
Downdload_path="C:/Users/abelmhai/Desktop/learning/challenge"
#init objects of our Data Base 
title=''
date=''
abstract=''
text_content=''
tags=''
external_content=''


# Here the first step : targeting and getting ready to walk across the data through the URL of the website

In [ ]:

#by using "urlopen" module from "request" library , we get into our website trought the URL
start_url=request.urlopen("https://www.capgemini.com/resources/?search_term=&filter_content_type=client-story&filter_label=&filter_research=")
#Once it's done , we pull out the Data by using the "BeautifulSoup" module , 
#this will help us to parse it and make it easier to navigate , search or modify the source code
start_page_content=bs4.BeautifulSoup(start_url.read())

#Number of pages that we have to cross (P.S : you need to know where the NÂ° is provided in the source code)
nb_pages= (start_page_content.find("div", { "class" : "pagination__current-page" }).text).split(' ')[3]


# the core of the method : analyse , spot and catch the data

In [ ]:

for page in range(1,int(nb_pages)+1):
    
    #according to which page we are , we recover the data from it
    url_list_resources = 'https://www.capgemini.com/resources/page/'+str(page)+'/?search_term=&filter_content_type=client-story&filter_label=&filter_research='
    page_list_resources_content=bs4.BeautifulSoup(request.urlopen(url_list_resources).read())
    #We retrieve in a list the tags containing the explicit links to the articles of the current page
    resources = page_list_resources_content.findAll("h3", { "class" : "card__media__title" })
    for resource in resources:
        i+=1
        #Across the list above , we gonna open and read each link to the articles
        next_url=resource.findNext('a').get('href')
        next_page_content=BeautifulSoup(request.urlopen(next_url).read())
        #then we start getting the Data referring to our objects that we need
        #(P.S : again , you need to study the source code to know which tag you need to catch and what information it refers )
        title=next_page_content.find("h1",{"class":"component__hero-subpage__title"}).text
        date=next_page_content.find('time').get('datetime')	
        #we use here the "try-except" mothod to handle errors during the scrap , and avoid that our script stop running
        try:
            #if the variable is null all the try clause stops and goes into the except clause
            abstract=next_page_content.find("div",{"class":"component__hero-inset--blog__lead"}).text
        except:
            abstract=''
            
        #we get the text of the delivery	
        text_content=next_page_content.find("div",{"class":"article-text"}).text 

        try:
            #same process here for "tags"
            tags=next_page_content.find("div",{"class":"component__hero-subpage__tags"}).text
        except:
            tags=''

        try:
            #here we get the links to our documentations (.pdf)
            pdf_download_link=next_page_content.find("div",{"class":"card__newsroom__download"}).findNext('a').get('href')
            #we recover the name of the file that will be used during the the export to our local storage
            pdf_filename = pdf_download_link.split('/')[-1]
            external_content=pdf_filename
            #start downloading
            request.urlretrieve(pdf_download_link,Downdload_path+'/'+pdf_filename)
        except:
            external_content=''
        
        #we build our database by containing all the objects by the right order
        DataBase.append([title,date,abstract,text_content,tags,external_content])
        print("Reference "+ str(i) + " scrapped")


# build and export the Database

In [ ]:

#We replace the "\n" by an empty value because it will give line breaks when exporting the data into csv file
for reference in DataBase:
    for element in reference:
        reference.replace("\n"," ")

##Exporting the database as a CSV file
df=pd.DataFrame(DataBase,columns=['Title','Date','Abstract','Text Content','Tags','External Content'])
df.to_csv("C:/Users/abelmhai/Desktop/learning/challenge/DB.csv", sep=";", encoding='utf-8')